In [2]:
! pip install transformers
# Transformers installation
!pip install transformers datasets
!pip install evaluate
!pip install jiwer

# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [3]:
import os
import numpy as np
import pandas as pd
import json


In [4]:
trainval_image_dir = os.path.join('/kaggle/input/coco-image-caption', 'train2014', 'train2014')
trainval_captions_dir = os.path.join('/kaggle/input/coco-image-caption', 'annotations_trainval2014', 'annotations')
test_image_dir = os.path.join('/kaggle/input/coco-image-caption', 'val2017', 'val2017')
test_captions_dir = os.path.join('/kaggle/input/coco-image-caption', 'annotations_trainval2017', 'annotations')

trainval_captions_filepath = os.path.join(trainval_captions_dir, 'captions_train2014.json')
test_captions_filepath = os.path.join(test_captions_dir, 'captions_val2017.json')

all_filepaths = np.array([os.path.join(trainval_image_dir, f) for f in os.listdir(trainval_image_dir)])
rand_indices = np.arange(len(all_filepaths))
np.random.shuffle(rand_indices)

split = int(len(all_filepaths)*0.8)

train_filepaths, valid_filepaths = all_filepaths[rand_indices[:split]], all_filepaths[rand_indices[split:]] 

print(f"Train dataset size: {len(train_filepaths)}")
print(f"Valid dataset size: {len(valid_filepaths)}")

with open(trainval_captions_filepath, 'r') as f:
    trainval_data = json.load(f)
    
trainval_captions_df = pd.json_normalize(trainval_data, "annotations")
trainval_captions_df["image_filepath"] = trainval_captions_df["image_id"].apply(
    lambda x: os.path.join(trainval_image_dir, 'COCO_train2014_'+format(x, '012d')+'.jpg')
)



train_captions_df = trainval_captions_df[trainval_captions_df["image_filepath"].isin(train_filepaths)]
valid_captions_df = trainval_captions_df[trainval_captions_df["image_filepath"].isin(valid_filepaths)]

with open(test_captions_filepath, 'r') as f:
    test_data = json.load(f)
    
test_captions_df = pd.json_normalize(test_data, "annotations")
test_captions_df["image_filepath"] = test_captions_df["image_id"].apply(
    lambda x: os.path.join(test_image_dir, format(x, '012d')+'.jpg')
)



Train dataset size: 66226
Valid dataset size: 16557


In [5]:
train_captions_df.head()

,image_id,id,caption,image_filepath
1,116100,67,A panoramic view of a kitchen and all of its a...,/kaggle/input/coco-image-caption/train2014/tra...
3,116100,148,A panoramic photo of a kitchen and dining room,/kaggle/input/coco-image-caption/train2014/tra...
4,379340,173,A graffiti-ed stop sign across the street from...,/kaggle/input/coco-image-caption/train2014/tra...
5,379340,188,A vandalized stop sign and a red beetle on the...,/kaggle/input/coco-image-caption/train2014/tra...
9,538480,288,A sink and a toilet inside a small bathroom.,/kaggle/input/coco-image-caption/train2014/tra...


In [6]:
print(valid_captions_df.info())
print(test_captions_df.info())
print(train_captions_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 82826 entries, 0 to 414109
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   image_id        82826 non-null  int64 
 1   id              82826 non-null  int64 
 2   caption         82826 non-null  object
 3   image_filepath  82826 non-null  object
dtypes: int64(2), object(2)
memory usage: 3.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25014 entries, 0 to 25013
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   image_id        25014 non-null  int64 
 1   id              25014 non-null  int64 
 2   caption         25014 non-null  object
 3   image_filepath  25014 non-null  object
dtypes: int64(2), object(2)
memory usage: 781.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 331287 entries, 1 to 414112
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype

In [7]:
train_captions_df['folder'] = 'train'
test_captions_df['folder'] = 'test'
valid_captions_df['folder'] = 'val'

# Combine the dataframes
combined_df = pd.concat([train_captions_df, test_captions_df, valid_captions_df], ignore_index=True)

# Save the combined dataframe if needed

# Display the combined dataframe
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439127 entries, 0 to 439126
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   image_id        439127 non-null  int64 
 1   id              439127 non-null  int64 
 2   caption         439127 non-null  object
 3   image_filepath  439127 non-null  object
 4   folder          439127 non-null  object
dtypes: int64(2), object(3)
memory usage: 16.8+ MB


/tmp/ipykernel_33/4285411469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_captions_df['folder'] = 'train'
/tmp/ipykernel_33/4285411469.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_captions_df['folder'] = 'val'


In [8]:
combined_df.head()

,image_id,id,caption,image_filepath,folder
0,116100,67,A panoramic view of a kitchen and all of its a...,/kaggle/input/coco-image-caption/train2014/tra...,train
1,116100,148,A panoramic photo of a kitchen and dining room,/kaggle/input/coco-image-caption/train2014/tra...,train
2,379340,173,A graffiti-ed stop sign across the street from...,/kaggle/input/coco-image-caption/train2014/tra...,train
3,379340,188,A vandalized stop sign and a red beetle on the...,/kaggle/input/coco-image-caption/train2014/tra...,train
4,538480,288,A sink and a toilet inside a small bathroom.,/kaggle/input/coco-image-caption/train2014/tra...,train


In [18]:
combined_df.folder.value_counts()

folder
train    331291
val       82822
test      25014
Name: count, dtype: int64

# Image captioning

In [23]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf")

from huggingface_hub import login


login(token=secret_value_0)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Load the Pokémon BLIP captions dataset

Use the 🤗 Dataset library to load a dataset that consists of {image-caption} pairs. To create your own image captioning dataset
in PyTorch, you can follow [this notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/GIT/Fine_tune_GIT_on_an_image_captioning_dataset.ipynb).

In [42]:
from datasets import Dataset, Image

train_captions_df['folder'] = 'train'
test_captions_df['folder'] = 'test'
valid_captions_df['folder'] = 'val'


/tmp/ipykernel_34/996047589.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_captions_df['folder'] = 'train'
/tmp/ipykernel_34/996047589.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_captions_df['folder'] = 'val'


In [43]:

# Assuming your DataFrame is called df

# loaded_dataset = Dataset.load_from_disk(dataset_name)


In [44]:
train_captions_df.drop(columns='folder' , inplace = True)
test_captions_df.drop(columns='folder' , inplace = True)
valid_captions_df.drop(columns='folder' , inplace = True)

/tmp/ipykernel_34/372525391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_captions_df.drop(columns='folder' , inplace = True)
/tmp/ipykernel_34/372525391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_captions_df.drop(columns='folder' , inplace = True)


In [9]:
train_captions_df.rename(columns ={"image_filepath": "image"} , inplace = True)
test_captions_df.rename(columns ={"image_filepath": "image"} , inplace = True)
valid_captions_df.rename(columns ={"image_filepath": "image"} , inplace = True)

/tmp/ipykernel_33/2716492222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_captions_df.rename(columns ={"image_filepath": "image"} , inplace = True)
/tmp/ipykernel_33/2716492222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_captions_df.rename(columns ={"image_filepath": "image"} , inplace = True)


In [10]:
train_captions_df.head()

,image_id,id,caption,image,folder
1,116100,67,A panoramic view of a kitchen and all of its a...,/kaggle/input/coco-image-caption/train2014/tra...,train
3,116100,148,A panoramic photo of a kitchen and dining room,/kaggle/input/coco-image-caption/train2014/tra...,train
4,379340,173,A graffiti-ed stop sign across the street from...,/kaggle/input/coco-image-caption/train2014/tra...,train
5,379340,188,A vandalized stop sign and a red beetle on the...,/kaggle/input/coco-image-caption/train2014/tra...,train
9,538480,288,A sink and a toilet inside a small bathroom.,/kaggle/input/coco-image-caption/train2014/tra...,train


In [14]:
from datasets import Dataset, Image , DatasetDict

In [15]:
# Assuming your dataset is already loaded and stored in the variable `dataset`

df_dict_train = train_captions_df.to_dict(orient="list")
df_dict_test = test_captions_df.to_dict(orient="list")
df_dict_val = valid_captions_df.to_dict(orient="list")

# Splitting the dataset based on the 'folder' column
train_dataset = Dataset.from_dict(df_dict_train).cast_column("image", Image())
val_dataset = Dataset.from_dict(df_dict_test).cast_column("image", Image())
test_dataset = Dataset.from_dict(df_dict_val).cast_column("image", Image())

# Creating a DatasetDict with the split datasets
split_datasets = DatasetDict({'train': train_dataset, 'val': val_dataset, 'test': test_dataset})

# Printing out the number of rows in each split
print("Train dataset size:", len(train_dataset))
print("Val dataset size:", len(val_dataset))
print("Test dataset size:", len(test_dataset))


Train dataset size: 331287
Val dataset size: 25014
Test dataset size: 82826


In [ ]:
import random

# Determine the desired size of the reduced training dataset
desired_train_size = int(len(train_dataset) * 0.03)  # Reduce the size by 60%

# Randomly sample examples from the original training dataset to create the reduced training dataset
reduced_train_dataset = train_dataset.shuffle(seed=42).select(range(desired_train_size))

In [ ]:
reduced_train_dataset

In [ ]:
reduced_train_dataset.push_to_hub("astro21/coco-caption-train-split-10k")

In [ ]:
# reduced_train_dataset.push_to_hub("astro21/coco-caption-train-split-33k")

Uploading the dataset shards:   0%|          | 0/11 [00:00<?, ?it/s]

Map:   0%|          | 0/3012 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Map:   0%|          | 0/3012 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Map:   0%|          | 0/3012 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Map:   0%|          | 0/3012 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

In [49]:
split_datasets.push_to_hub("astro21/coco-caption-split")

Uploading the dataset shards:   0%|          | 0/99 [00:00<?, ?it/s]

Map:   0%|          | 0/3347 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Map:   0%|          | 0/3347 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

...

Map:   0%|          | 0/3312 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

Map:   0%|          | 0/3312 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/34 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/astro21/coco-caption-split/commit/0508238049ab4e3a77326cdf992967a74d50f826', commit_message='Upload dataset (part 00002-of-00003)', commit_description='', oid='0508238049ab4e3a77326cdf992967a74d50f826', pr_url=None, pr_revision=None, pr_num=None)